In [1]:
from datasets import load_dataset
import json 
import os

ds = load_dataset("deepmind/code_contests", split="train")
print(ds)

/opt/conda/envs/rllm-xiao/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since deepmind/code_contests couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/xiaoxiang/.cache/huggingface/datasets/deepmind___code_contests/default/0.0.0/802411c3010cb00d1b05bad57ca77365a3c699d6 (last modified on Fri Feb 28 07:05:53 2025).


Dataset({
    features: ['name', 'description', 'public_tests', 'private_tests', 'generated_tests', 'source', 'difficulty', 'solutions', 'incorrect_solutions', 'cf_contest_id', 'cf_index', 'cf_points', 'cf_rating', 'cf_tags', 'is_description_translated', 'untranslated_description', 'time_limit', 'memory_limit_bytes', 'input_file', 'output_file'],
    num_rows: 13328
})


In [2]:
TEMPLATE = """
{problem}

Time Limit: {time_limit}
Memory Limit: {memory_limit} Bytes
"""
from pprint import pprint
dataset = []
for entry in ds:
    difficulty = entry["difficulty"] # TODO: understand how the difficulty is calculated and convert the difficulty to a number from 1-10 or 0 for unknown        

    if not (difficulty >= 2 and difficulty <= 5) and not (difficulty >=10):
        continue

    input_tests = entry["public_tests"]['input'] + entry["private_tests"]['input']
    output_tests = entry["public_tests"]['output'] + entry["private_tests"]['output']
    
    if len(input_tests) <= 1:
        continue
    assert len(input_tests) == len(output_tests)
    
    full_tests = {
        "input": input_tests,
        "output": output_tests,
    }
    new_entry = {
        "problem": TEMPLATE.format(problem=entry["description"], time_limit=entry["time_limit"], memory_limit=entry["memory_limit_bytes"]),
        "tests": full_tests,
    }
    dataset.append(new_entry)

print(len(dataset))

3576


In [3]:

output_dir = os.path.abspath("../../train/code")
output_file = os.path.join(output_dir, "code_contests_solution.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)

In [ ]:
ds = load_dataset("deepmind/code_contests", split="test")


dataset = []
for entry in ds:
    difficulty = entry["difficulty"] # TODO: understand how the difficulty is calculated and convert the difficulty to a number from 1-10 or 0 for unknown

    input_tests = entry["public_tests"]['input'] + entry["private_tests"]['input']
    output_tests = entry["public_tests"]['output'] + entry["private_tests"]['output']
    
    full_tests = {
        "input": input_tests,
        "output": output_tests,
    }
    
    new_entry = {
        "problem": TEMPLATE.format(problem=entry["description"], time_limit=entry["time_limit"], memory_limit=entry["memory_limit_bytes"]),
        "tests": full_tests
    }
    dataset.append(new_entry)

print(len(dataset))

output_dir = os.path.abspath("../../test/code")
output_file = os.path.join(output_dir, "code_contests.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)

165
